# Actividad práctica de Spark - Striccoli Juan Gabriel

## Importar las bibliotecas necesarias y crear una instancia de SparkSession

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("PracticaSpark") \
    .config("spark.executor.memory", "2g") \
    .config("spark.executor.cores", "2") \
    .getOrCreate()

## Ejecución de los ejercicios requeridos

### 1. Crear un DataFrame con datos de personas, que incluya nombre, edad y ciudad de residencia

In [2]:
# Primero defino el Schema
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([
    StructField("Nombre", StringType(), True),
    StructField("Edad", IntegerType(), True),
    StructField("Ciudad", StringType(), True)
])

# Luego creo el DataFrame
data = [("Alice", 25, "New York"),
        ("Bob", 30, "Los Angeles"),
        ("Charlie", 22, "Chicago")]

df = spark.createDataFrame(data, schema=schema)

In [3]:
df.printSchema()

root
 |-- Nombre: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Ciudad: string (nullable = true)



In [4]:
df.show(3)

+-------+----+-----------+
| Nombre|Edad|     Ciudad|
+-------+----+-----------+
|  Alice|  25|   New York|
|    Bob|  30|Los Angeles|
|Charlie|  22|    Chicago|
+-------+----+-----------+



### 2. Mostrar solo los nombres de las personas del DataFrame

In [5]:
df.select("Nombre").show()

+-------+
| Nombre|
+-------+
|  Alice|
|    Bob|
|Charlie|
+-------+



### 3. Filtrar personas cuya edad sea mayor o igual a 25

In [6]:
df.filter(df.Edad >= 25).show()

+------+----+-----------+
|Nombre|Edad|     Ciudad|
+------+----+-----------+
| Alice|  25|   New York|
|   Bob|  30|Los Angeles|
+------+----+-----------+



### 4. Agregar una nueva columna "Pais" con un valor constante para todas las filas

In [7]:
from pyspark.sql.functions import lit

df = df.withColumn("Pais", lit("Argentina"))

In [8]:
df.show(3)

+-------+----+-----------+---------+
| Nombre|Edad|     Ciudad|     Pais|
+-------+----+-----------+---------+
|  Alice|  25|   New York|Argentina|
|    Bob|  30|Los Angeles|Argentina|
|Charlie|  22|    Chicago|Argentina|
+-------+----+-----------+---------+



### 5. Calcular el promedio de edad de todas las personas

In [9]:
from pyspark.sql.functions import avg

df.select(avg("Edad")).show()

+------------------+
|         avg(Edad)|
+------------------+
|25.666666666666668|
+------------------+



### 6. Ordenar el DataFrame por edad en orden descendente

In [10]:
df.orderBy(df.Edad.desc()).show()

+-------+----+-----------+---------+
| Nombre|Edad|     Ciudad|     Pais|
+-------+----+-----------+---------+
|    Bob|  30|Los Angeles|Argentina|
|  Alice|  25|   New York|Argentina|
|Charlie|  22|    Chicago|Argentina|
+-------+----+-----------+---------+



### 7. Agrupar por ciudad y calcular la cantidad de personas en cada ciudad

In [11]:
df.groupBy("Ciudad").count().show()

+-----------+-----+
|     Ciudad|count|
+-----------+-----+
|   New York|    1|
|Los Angeles|    1|
|    Chicago|    1|
+-----------+-----+



### 8. Renombrar la columna "Nombre" a "NombreCompleto"

In [12]:
df = df.withColumnRenamed("Nombre", "NombreCompleto")

In [13]:
df.show(3)

+--------------+----+-----------+---------+
|NombreCompleto|Edad|     Ciudad|     Pais|
+--------------+----+-----------+---------+
|         Alice|  25|   New York|Argentina|
|           Bob|  30|Los Angeles|Argentina|
|       Charlie|  22|    Chicago|Argentina|
+--------------+----+-----------+---------+



### 9. Eliminar la columna "Edad" del DataFrame

In [14]:
df2 = df.drop("Edad")

In [15]:
df2.show(3)

+--------------+-----------+---------+
|NombreCompleto|     Ciudad|     Pais|
+--------------+-----------+---------+
|         Alice|   New York|Argentina|
|           Bob|Los Angeles|Argentina|
|       Charlie|    Chicago|Argentina|
+--------------+-----------+---------+



### 10. Realizar una consulta SQL en el DataFrame para seleccionar personas mayores de 20 años

In [16]:
df.createOrReplaceTempView("personas")
result = spark.sql("SELECT * FROM personas WHERE Edad > 20")
result.show()

+--------------+----+-----------+---------+
|NombreCompleto|Edad|     Ciudad|     Pais|
+--------------+----+-----------+---------+
|         Alice|  25|   New York|Argentina|
|           Bob|  30|Los Angeles|Argentina|
|       Charlie|  22|    Chicago|Argentina|
+--------------+----+-----------+---------+



### 11. Calcular la suma total de todas las edades

In [17]:
from pyspark.sql.functions import sum

df.select(sum("Edad")).show()

+---------+
|sum(Edad)|
+---------+
|       77|
+---------+



### 12. Calcular la edad mínima y máxima de todas las personas.

In [18]:
from pyspark.sql.functions import min, max

df.select(min("Edad"), max("Edad")).show()

+---------+---------+
|min(Edad)|max(Edad)|
+---------+---------+
|       22|       30|
+---------+---------+



### 13. Filtrar personas cuya ciudad de residencia sea "Chicago" y edad sea menor de 30

In [19]:
df.filter((df.Ciudad == "Chicago") & (df.Edad < 30)).show()

+--------------+----+-------+---------+
|NombreCompleto|Edad| Ciudad|     Pais|
+--------------+----+-------+---------+
|       Charlie|  22|Chicago|Argentina|
+--------------+----+-------+---------+



### 14. Agregar una nueva columna "EdadDuplicada" que contenga el doble de la edad

In [20]:
df = df.withColumn("EdadDuplicada", df.Edad * 2)

In [21]:
df.show(3)

+--------------+----+-----------+---------+-------------+
|NombreCompleto|Edad|     Ciudad|     Pais|EdadDuplicada|
+--------------+----+-----------+---------+-------------+
|         Alice|  25|   New York|Argentina|           50|
|           Bob|  30|Los Angeles|Argentina|           60|
|       Charlie|  22|    Chicago|Argentina|           44|
+--------------+----+-----------+---------+-------------+



### 15. Convertir todas las edades en años a meses

In [22]:
df = df.withColumn("EdadEnMeses", df.Edad * 12)

In [23]:
df.show(3)

+--------------+----+-----------+---------+-------------+-----------+
|NombreCompleto|Edad|     Ciudad|     Pais|EdadDuplicada|EdadEnMeses|
+--------------+----+-----------+---------+-------------+-----------+
|         Alice|  25|   New York|Argentina|           50|        300|
|           Bob|  30|Los Angeles|Argentina|           60|        360|
|       Charlie|  22|    Chicago|Argentina|           44|        264|
+--------------+----+-----------+---------+-------------+-----------+



### 16. Contar el número total de personas en el DataFrame

In [26]:
total_personas = df.count()

print("Número total de personas es:", total_personas)

Número total de personas es: 3


### 17. Filtrar personas cuya edad sea un número par

In [27]:
df.filter(df.Edad % 2 == 0).show()

+--------------+----+-----------+---------+-------------+-----------+
|NombreCompleto|Edad|     Ciudad|     Pais|EdadDuplicada|EdadEnMeses|
+--------------+----+-----------+---------+-------------+-----------+
|           Bob|  30|Los Angeles|Argentina|           60|        360|
|       Charlie|  22|    Chicago|Argentina|           44|        264|
+--------------+----+-----------+---------+-------------+-----------+



### 18. Calcular la cantidad de personas por rango de edades (0-20, 21-40, 41-60, 61+)

In [28]:
from pyspark.sql.functions import when

df = df.withColumn("RangoEdades",
                   when((df.Edad >= 0) & (df.Edad <= 20), "0-20")
                   .when((df.Edad >= 21) & (df.Edad <= 40), "21-40")
                   .when((df.Edad >= 41) & (df.Edad <= 60), "41-60")
                   .otherwise("61+"))

df.groupBy("RangoEdades").count().show()

+-----------+-----+
|RangoEdades|count|
+-----------+-----+
|      21-40|    3|
+-----------+-----+



### 19. Contar cuántas personas tienen el mismo nombre

In [29]:
df.groupBy("NombreCompleto").count().show()

+--------------+-----+
|NombreCompleto|count|
+--------------+-----+
|         Alice|    1|
|           Bob|    1|
|       Charlie|    1|
+--------------+-----+



In [32]:
#La otra opción es mostrando únicamente aquellos que tienen duplicado el nombre más de 1 vez
from pyspark.sql.functions import col
grupoNombre = df.groupBy("NombreCompleto").count()
duplicados = grupoNombre.filter(col("count") > 1).show()

+--------------+-----+
|NombreCompleto|count|
+--------------+-----+
+--------------+-----+



### 20. Concatenar las columnas "Nombre" y "Ciudad" en una nueva columna llamada "InformacionPersonal"

In [33]:
df = df.withColumn("InformacionPersonal", df.NombreCompleto + ", " + df.Ciudad)

df.show()

+--------------+----+-----------+---------+-------------+-----------+-----------+-------------------+
|NombreCompleto|Edad|     Ciudad|     Pais|EdadDuplicada|EdadEnMeses|RangoEdades|InformacionPersonal|
+--------------+----+-----------+---------+-------------+-----------+-----------+-------------------+
|         Alice|  25|   New York|Argentina|           50|        300|      21-40|               NULL|
|           Bob|  30|Los Angeles|Argentina|           60|        360|      21-40|               NULL|
|       Charlie|  22|    Chicago|Argentina|           44|        264|      21-40|               NULL|
+--------------+----+-----------+---------+-------------+-----------+-----------+-------------------+

